In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from keras.models import Model
from keras.layers import LSTM, Activation, Dense, Dropout, Input, Embedding
from keras.optimizers import RMSprop
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence
from keras.utils import to_categorical
from keras.callbacks import EarlyStopping
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Conv1D, MaxPooling1D

In [2]:
df = pd.read_csv('ecom-user-churn-data.csv')
df.head()

,visitorid,ses_rec,ses_rec_avg,ses_rec_sd,ses_rec_cv,user_rec,ses_n,ses_n_r,int_n,int_n_r,...,int_cat16_n,int_cat17_n,int_cat18_n,int_cat19_n,int_cat20_n,int_cat21_n,int_cat22_n,int_cat23_n,int_cat24_n,target_class
0,89,23.0,16.0,0.0,0.0,39.0,2.0,0.051282,3.0,1.500000,...,0,0,0,0,3,0,0,0,0,1
1,251,23.0,61.0,0.0,0.0,85.0,2.0,0.023529,2.0,1.000000,...,0,0,0,0,0,0,1,0,0,1
2,270,14.0,0.1,0.3,3.0,21.0,11.0,0.523810,13.0,1.181818,...,0,0,0,0,0,0,13,0,0,1
3,298,11.0,13.0,0.0,0.0,24.0,2.0,0.083333,4.0,2.000000,...,0,0,0,3,1,0,0,0,0,1
4,474,16.0,26.0,0.0,0.0,43.0,2.0,0.046512,2.0,1.000000,...,2,0,0,0,0,0,0,0,0,1


In [4]:
df.shape

(49358, 49)

In [5]:
columns_to_remove = df.columns[df.columns.str.startswith('int_cat') & df.columns.str.endswith('_n')]

df= df.drop(columns=columns_to_remove)


In [6]:
df.shape

(49358, 26)

In [8]:
df.columns

Index(['visitorid', 'ses_rec', 'ses_rec_avg', 'ses_rec_sd', 'ses_rec_cv',
       'user_rec', 'ses_n', 'ses_n_r', 'int_n', 'int_n_r', 'tran_n',
       'tran_n_r', 'rev_sum', 'rev_sum_r', 'major_spend_r', 'int_cat_n_avg',
       'int_itm_n_avg', 'ses_mo_avg', 'ses_mo_sd', 'ses_ho_avg', 'ses_ho_sd',
       'ses_wknd_r', 'ses_len_avg', 'time_to_int', 'time_to_tran',
       'target_class'],
      dtype='object')

In [9]:
X = df[['int_n_r','ses_len_avg','time_to_tran','tran_n_r','user_rec']]
Y = df.target_class
le = LabelEncoder()
Y = le.fit_transform(Y)
Y = Y.reshape(-1,1)

In [10]:
X_train,X_test,Y_train,Y_test = train_test_split(X,Y,test_size=0.15)

In [31]:
max_features = None
maxlen = 100
embedding_size = None
# Convolution
kernel_size = 5
filters = 64
pool_size = 4
# LSTM
lstm_output_size = 70
# Training
batch_size = 30
epochs = 5

In [45]:
#Without Padding
maxlen = 5  # Assuming your input sequences have a maximum length of 5

# Other parameters remain the same
kernel_size = 5
filters = 64
pool_size = 2
lstm_output_size = 70
batch_size = 30
epochs = 5


In [32]:
model = Sequential()
model.add(Conv1D(filters=filters,
                 kernel_size=kernel_size,
                 padding='valid',
                 activation='relu',
                 strides=1,
                 input_shape=(maxlen, 1)))  # Input shape for Conv1D layer
model.add(MaxPooling1D(pool_size=pool_size))
model.add(LSTM(lstm_output_size))
model.add(Dense(1, activation='sigmoid'))

# Compile the model
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

# Display model summary
model.summary()

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d_4 (Conv1D)           (None, 96, 64)            384       
                                                                 
 max_pooling1d_4 (MaxPoolin  (None, 24, 64)            0         
 g1D)                                                            
                                                                 
 lstm_3 (LSTM)               (None, 70)                37800     
                                                                 
 dense_3 (Dense)             (None, 1)                 71        
                                                                 
Total params: 38255 (149.43 KB)
Trainable params: 38255 (149.43 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [47]:
#Without Padding
model = Sequential()
model.add(Conv1D(filters=filters,
                 kernel_size=kernel_size,
                 padding='valid',
                 activation='relu',
                 strides=1,
                 input_shape=(maxlen, 1)))  # Input shape for Conv1D layer
# Removed MaxPooling1D layer
model.add(LSTM(lstm_output_size))
model.add(Dense(1, activation='sigmoid'))

# Compile the model
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

# Display model summary
model.summary()

Model: "sequential_10"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d_9 (Conv1D)           (None, 1, 64)             384       
                                                                 
 lstm_4 (LSTM)               (None, 70)                37800     
                                                                 
 dense_4 (Dense)             (None, 1)                 71        
                                                                 
Total params: 38255 (149.43 KB)
Trainable params: 38255 (149.43 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [33]:
X_train_array = X_train.values  # Convert DataFrame to NumPy array
maxlen = 100  # Desired sequence length
X_train_padded = np.zeros((len(X_train_array), maxlen, 1))  # Create an array for padded sequences

# Copy the values of X_train_array to X_train_padded with padding/truncation
for i, seq in enumerate(X_train_array):
    seq_len = len(seq)
    if seq_len >= maxlen:
        X_train_padded[i, :, :] = seq[:maxlen].reshape(-1, 1)  # Truncate if sequence is longer than maxlen
    else:
        X_train_padded[i, :seq_len, :] = seq.reshape(-1, 1)

In [34]:
model.fit(X_train_padded, Y_train,
          batch_size=batch_size,
          epochs=epochs,
          validation_split=0.2)

Epoch 1/5
1119/1119 [==============================] - 27s 21ms/step - loss: 0.3590 - accuracy: 0.8859 - val_loss: 0.3585 - val_accuracy: 0.8845
Epoch 2/5
1119/1119 [==============================] - 23s 21ms/step - loss: 0.3557 - accuracy: 0.8858 - val_loss: 0.3580 - val_accuracy: 0.8845
Epoch 3/5
1119/1119 [==============================] - 24s 21ms/step - loss: 0.3473 - accuracy: 0.8859 - val_loss: 0.3501 - val_accuracy: 0.8845
Epoch 4/5
1119/1119 [==============================] - 23s 21ms/step - loss: 0.3452 - accuracy: 0.8859 - val_loss: 0.3484 - val_accuracy: 0.8845
Epoch 5/5
1119/1119 [==============================] - 24s 21ms/step - loss: 0.3447 - accuracy: 0.8859 - val_loss: 0.3472 - val_accuracy: 0.8845


In [48]:
#Without Padding
model.fit(X_train, Y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    validation_split=0.2)

Epoch 1/5
1119/1119 [==============================] - 9s 5ms/step - loss: 0.3479 - accuracy: 0.8854 - val_loss: 0.3465 - val_accuracy: 0.8844
Epoch 2/5
1119/1119 [==============================] - 5s 5ms/step - loss: 0.3444 - accuracy: 0.8858 - val_loss: 0.3470 - val_accuracy: 0.8845
Epoch 3/5
1119/1119 [==============================] - 5s 4ms/step - loss: 0.3438 - accuracy: 0.8859 - val_loss: 0.3473 - val_accuracy: 0.8845
Epoch 4/5
1119/1119 [==============================] - 5s 4ms/step - loss: 0.3434 - accuracy: 0.8859 - val_loss: 0.3468 - val_accuracy: 0.8845
Epoch 5/5
1119/1119 [==============================] - 5s 5ms/step - loss: 0.3435 - accuracy: 0.8859 - val_loss: 0.3487 - val_accuracy: 0.8845


In [36]:
X_test_array = X_test.values  # Convert DataFrame to NumPy array
maxlen = 100  # Desired sequence length
X_test_padded = np.zeros((len(X_test_array), maxlen, 1))  # Create an array for padded sequences

# Copy the values of X_test_array to X_test_padded with padding/truncation
for i, seq in enumerate(X_test_array):
    seq_len = len(seq)
    if seq_len >= maxlen:
        X_test_padded[i, :, :] = seq[:maxlen].reshape(-1, 1)  # Truncate if sequence is longer than maxlen
    else:
        X_test_padded[i, :seq_len, :] = seq.reshape(-1, 1)

In [38]:
accr=model.evaluate(X_test_padded, Y_test)

232/232 [==============================] - 2s 9ms/step - loss: 0.3460 - accuracy: 0.8855


In [49]:
#Without Padding
loss, accuracy = model.evaluate(X_test, Y_test)
print(f'Test Loss: {loss}, Test Accuracy: {accuracy}')

232/232 [==============================] - 1s 2ms/step - loss: 0.3467 - accuracy: 0.8855
Test Loss: 0.3466593325138092, Test Accuracy: 0.8854672908782959


In [39]:
print('Test set\n  Loss: {:0.3f}\n  Accuracy: {:0.3f}'.format(accr[0],accr[1]))

Test set
  Loss: 0.346
  Accuracy: 0.885
